In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


In [2]:
def dataclean(zillow_yvalues_data,census_data,date):
    # takes in the zillow data with all the years and acs census data of two years before 
    # year to predict and date formatted '2020-09' and joins the census on zillow and cleans 
    
    
    zillow_yvalues_data_yearsubsect=zillow_yvalues_data[['RegionName','City','State',date]]
    fulldata=zillow_yvalues_data_yearsubsect.merge(census_data,how='left',left_on='RegionName', right_on='geo_id' ).drop(['geo_id'],axis=1)
    fulldata=fulldata[fulldata[date].isna()!=1]
    #drop column if >20% are nas 
    cols_to_drop=['do_date']
    for i in fulldata.columns:

        if sum(fulldata[i].isna())/len(fulldata)> .2:
            cols_to_drop.append(i)
    print(cols_to_drop)
    fulldata=fulldata.drop(cols_to_drop,axis=1)
    #drop rows with more than 50% nas
    #in this jan2020was one row
    fulldata=fulldata.loc[fulldata.isna().sum(axis=1)<.5*len(fulldata.columns)]
    #num of missing values in columns
    countofcols=0
    cols_with_na=[]
    for i in fulldata.columns:
        if i in cols_to_drop:
            print(i)
        elif sum(fulldata[i].isna())>0:
#             print('Column:',i)
#             print('Num of NAs:',sum(fulldata[i].isna()))
#             print('Num of 0s:',sum(fulldata[i]==0))
            cols_with_na.append(i)
            countofcols+=1
    #fill in nas with avrg of the column for the city 
    for i in cols_with_na:
        for indexofna in (list(fulldata[i][fulldata[i].isna()].index)):

            if bool(~(fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()==fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean())):
                fulldata.loc[indexofna,i]=fulldata[fulldata.State==fulldata.loc[indexofna,'State']][i].mean()
                
            else:
                fulldata.loc[indexofna,i]=fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()
                
    
    return fulldata

In [3]:
#fulldata=dataclean(zillow_yvalues_data,census_data,date)

# Assembling training data w yvals from 2011-2017 and before 
-means census data from 2015 and before 


In [4]:
def assemble_train_test_predict(years_out=1,year_gap_in_data=2):
    
    df = pd.read_csv('Zip_Zri_MultiFamilyResidenceRental-2.csv')
    zillow_yvalues_data=pd.merge(df.iloc[:,0:7], df.iloc[:,8:].interpolate(axis=1), left_index=True, right_index=True)
    genericfilepath='acs_data/acs20??.csv'
    census_data_years=list(range(11,19,1))
    years=list(range(13,21,1))
    years_for_train=years[:-years_out]
    years_for_pred=years[-years_out:]
    census_data_years_train=census_data_years[:-years_out]
    print(years_for_train)
    print(census_data_years_train)
    olddatedata=pd.DataFrame()
    for i in years_for_train:
        print(i)
        tempfilepath=genericfilepath.replace('??', str(i-year_gap_in_data))
        yearnumber=str(i)
        if len(yearnumber)<2:
            yearnumber='0'+yearnumber
        date='20'+yearnumber+'-09'
#         print('date',date)
#         print('tempfilepath',tempfilepath)
        try:
            census_data= pd.read_csv(tempfilepath)
            datedata=dataclean(zillow_yvalues_data,census_data,date)
            datedata['RegionName']=datedata['RegionName'].astype(str)+'-'+date
            datedata=datedata.rename(columns={date: 'Rent'})
            olddatedata=pd.concat([datedata,olddatedata], ignore_index=True, sort=False)
        except:
            p=1
    
    
    fulldata=olddatedata
    cols_to_drop=[]
    for i in fulldata.columns:
        if sum(fulldata[i].isna())/len(fulldata)> .2:
            cols_to_drop.append(i)
    fulldata=fulldata.drop(cols_to_drop,axis=1)
    
    
#     print('\n\npred\n\n')
    
    #assemble prediction dataset
    olddatedata=pd.DataFrame()
    for i in years_for_pred:
#         print(i)
        tempfilepath=genericfilepath.replace('??', str(i-year_gap_in_data))
        yearnumber=str(i)
        if len(yearnumber)<2:
            yearnumber='0'+yearnumber
        date='20'+yearnumber+'-09'
        if date=='2020-09':
            date='2020-01'
#         print('date',date)
#         print('tempfilepath',tempfilepath)
        
        census_data= pd.read_csv(tempfilepath)
        datedata=dataclean(zillow_yvalues_data,census_data,date)
        datedata['RegionName']=datedata['RegionName'].astype(str)+'-'+date
        datedata=datedata.rename(columns={date: 'Rent'})
#         newfilename='Data_acsPREDICT20'+yearnumber+'.csv'
#         datedata.to_csv(newfilename)
        olddatedata=pd.concat([datedata,olddatedata], ignore_index=True, sort=False)
        
    
    
    preddata=olddatedata
    cols_to_drop=[]
    for i in preddata.columns:
        if sum(preddata[i].isna())/len(preddata)> .2:
            cols_to_drop.append(i)
    preddata=preddata.drop(cols_to_drop,axis=1)
    
    
    #ensures prediciton and training set have same cols 
    cols_in_common=list(fulldata.columns & preddata.columns)
    print(len(preddata.columns))
    print(len(cols_in_common))
    preddata=preddata[cols_in_common]
    fulldata=fulldata[cols_in_common]

    
    
#     #train test split
    tdata_train, tdata_test = train_test_split(fulldata, test_size=0.33, random_state=100)
    tdata_train.to_csv('Data_acsTRAIN.csv')
    tdata_test.to_csv('Data_acsTEST.csv')
    preddata.to_csv('Data_acsPREDICT.csv')
    
    return preddata
    


In [5]:
assemble_train_test_predict(years_out=3,year_gap_in_data=2)


[13, 14, 15, 16, 17]
[11, 12, 13, 14, 15]
13
['do_date', 'associates_degree', 'bachelors_degree', 'high_school_diploma', 'less_one_year_college', 'masters_degree', 'one_year_more_college', 'pop_25_years_over']
14
['do_date']
15
['do_date']
16
['do_date']
17
['do_date', 'geoid']
['do_date', 'pop_5_years_over', 'speak_only_english_at_home', 'speak_spanish_at_home', 'speak_spanish_at_home_low_english', 'pop_15_and_over', 'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced']
['do_date', 'pop_5_years_over', 'speak_only_english_at_home', 'speak_spanish_at_home', 'speak_spanish_at_home_low_english', 'pop_15_and_over', 'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced']
['do_date', 'pop_5_years_over', 'pop_15_and_over', 'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced', 'speak_only_english_at_home', 'speak_spanish_at_home', 'speak_spanish_at_home_low_english']
232
232


,RegionName,City,State,Rent,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,...,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,hispanic_any_race
0,10025-2020-01,New York,NY,3676.0,22248.0,19662.0,0.0,1499.0,7255.0,2293.0,...,5748.0,1721.0,15509.0,5086.0,70558.0,6257.0,5283.0,1646.0,42055.0,21736.0
1,60657-2020-01,Chicago,IL,1731.0,25196.0,11428.0,1958.0,893.0,3159.0,1415.0,...,2105.0,1077.0,11793.0,3242.0,53263.0,8384.0,4818.0,1640.0,46468.0,4901.0
2,10023-2020-01,New York,NY,3561.0,20157.0,13536.0,1941.0,853.0,3194.0,1238.0,...,2914.0,915.0,15213.0,2738.0,50726.0,3359.0,2266.0,979.0,31688.0,6562.0
3,77494-2020-01,Katy,TX,1286.0,4825.0,30517.0,2009.0,324.0,1358.0,495.0,...,6213.0,2638.0,13063.0,9765.0,69744.0,5312.0,9007.0,2701.0,47526.0,24895.0
4,60614-2020-01,Chicago,IL,1916.0,21566.0,12018.0,1964.0,626.0,2819.0,1105.0,...,2333.0,776.0,11520.0,2448.0,50765.0,5092.0,4130.0,1529.0,43641.0,4317.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291,2142-2018-09,Cambridge,MA,2994.0,709.0,528.0,1998.0,45.0,302.0,48.0,...,19.0,6.0,743.0,39.0,1939.0,124.0,65.0,324.0,1402.0,217.0
5292,18232-2018-09,Lansford,PA,774.0,786.0,880.0,1939.0,23.0,141.0,54.0,...,1093.0,130.0,139.0,259.0,2757.0,164.0,102.0,289.0,1490.0,284.0
5293,75202-2018-09,Dallas,TX,1650.0,1112.0,305.0,1969.0,4.0,198.0,43.0,...,100.0,48.0,236.0,201.0,1781.0,56.0,72.0,329.0,1604.0,355.0
5294,60301-2018-09,Oak Park,IL,1535.0,963.0,436.0,1971.0,33.0,320.0,82.0,...,131.0,57.0,435.0,255.0,1900.0,157.0,161.0,103.0,1271.0,311.0
